# جمع بيانات من Reddit وFacebook وInstagram حول الجهات الحكومية في قطر

In [1]:

!pip install praw googletrans==4.0.0-rc1 langdetect pandas openpyxl google-search-results


In [ ]:

import praw
import pandas as pd
from langdetect import detect
from googletrans import Translator
from datetime import datetime

translator = Translator()

reddit = praw.Reddit(
    client_id='client_id',
    client_secret='client_secret,
    user_agent='user_agent'
)

keywords = {
    "وزارة التعليم": ["وزارة التعليم", "التعليم في قطر", "ministry of education qatar"],
    "وزارة الصحة": ["وزارة الصحة", "الصحة في قطر", "MOPH Qatar"],
    "وزارة التجارة والصناعة": ["وزارة التجارة", "التجارة قطر", "ministry of commerce qatar"],
    "وزارة المواصلات": ["وزارة المواصلات", "mots qatar"],
    "وزارة البلدية": ["وزارة البلدية", "municipality qatar"],
    "أشغال": ["هيئة الأشغال", "ashghal", "ashghal qatar"]
}

data = []

for entity, terms in keywords.items():
    for term in terms:
        for post in reddit.subreddit("all").search(term, limit=50):
            try:
                text = post.title + "\n" + post.selftext
                lang = detect(text)
                ar_text = text if lang == 'ar' else translator.translate(text, dest='ar').text
                en_text = text if lang == 'en' else translator.translate(text, dest='en').text
                data.append({
                    "ID": post.id,
                    "Source": "Reddit",
                    "Entity": entity,
                    "Arabic Post": ar_text,
                    "English": en_text,
                    "Original Lang": lang,
                    "Post Type": "Post",
                    "Hashtag": ', '.join([t for t in text.split() if t.startswith("#")]),
                    "Date": datetime.fromtimestamp(post.created_utc),
                    "Link": f"https://www.reddit.com{post.permalink}"
                })
            except:
                continue

df = pd.DataFrame(data)
df.to_excel("reddit_qatar_feedback.xlsx", index=False)
print("✅ Reddit data saved.")


✅ Reddit data saved.


In [ ]:
from serpapi import GoogleSearch
import pandas as pd
from langdetect import detect
from googletrans import Translator
import time

translator = Translator()

# أدخل مفتاح SerpAPI الخاص بك هنا
SERPAPI_API_KEY = "SERPAPI_API_KEY"

entities = {
    "وزارة التعليم قطر": [" وزارة التعليم قطر site:facebook.com", "وزارة التعليم قطر  site:instagram.com","وزارة التعليم قطر  site:twitter.com"],
    " وزارة الصحة قطر": ["وزارة الصحة قطر site:facebook.com", "وزارة الصحة قطر site:instagram.com","وزارة الصحة قطر site:twitter.com"],
    "وزارة التجارة والصناعة قطر": ["وزارة التجارة والصناعة قطر site:facebook.com", "وزارة التجارة والصناعة قطر site:instagram.com","وزارة التجارة والصناعة قطر site:twitter.com"],
    "وزارة المواصلات قطر": ["وزارة المواصلات قطر site:facebook.com", "وزارة المواصلات قطر site:instagram.com","وزارة المواصلات قطر site:twitter.com"],
    "وزارة البلدية قطر": ["وزارة البلدية قطر site:facebook.com", "وزارة البلدية قطر site:instagram.com","وزارة البلدية قطر site:twitter.com"],
    "أشغال قطر": [" هيئة الأشغال قطر site:facebook.com", "ashghal qatar site:instagram.com","ashghal qatar site:twitter.com"]
}

results = []

for entity, queries in entities.items():
    for query in queries:
        print(f"🔍 Searching Google for: {query}")
        params = {
            "engine": "google",
            "q": query,
            "hl": "ar",
            "num": 50,
            "api_key": SERPAPI_API_KEY,
            "tbs": "cdr:1,cd_min:1/1/2020,cd_max:12/31/2025"
        }
        search = GoogleSearch(params)
        try:
            results_data = search.get_dict()
            for r in results_data.get("organic_results", []):
                title = r.get("title", "")
                snippet = r.get("snippet", "")
                url = r.get("link", "")
                text = title + " " + snippet
                lang = detect(text)
                ar = text if lang == "ar" else translator.translate(text, dest='ar').text
                en = text if lang == "en" else translator.translate(text, dest='en').text
                results.append({
                    "ID": url.split("/")[-1][:15],
                    "Source": "Facebook/Instagram (via Google)",
                    "Entity": entity,
                    "Arabic Post": ar,
                    "English": en,
                    "Original Lang": lang,
                    "Post Type": "Post",
                    "Hashtag": ', '.join([t for t in text.split() if t.startswith("#")]),
                    "Date": "",
                    "Link": url
                })
            time.sleep(1)
        except Exception as e:
            print(f"❌ Error for query {query}: {e}")

# حفظ النتائج
df = pd.DataFrame(results)
df.to_excel("fb_ig_from_google.xlsx", index=False)
print("✅ Data saved to fb_ig_from_google.xlsx")



🔍 Searching Google for:  وزارة التعليم قطر site:facebook.com
🔍 Searching Google for: وزارة التعليم قطر  site:instagram.com
🔍 Searching Google for: وزارة التعليم قطر  site:twitter.com
🔍 Searching Google for: وزارة الصحة قطر site:facebook.com
🔍 Searching Google for: وزارة الصحة قطر site:instagram.com
🔍 Searching Google for: وزارة الصحة قطر site:twitter.com
🔍 Searching Google for: وزارة التجارة والصناعة قطر site:facebook.com
🔍 Searching Google for: وزارة التجارة والصناعة قطر site:instagram.com
🔍 Searching Google for: وزارة التجارة والصناعة قطر site:twitter.com
🔍 Searching Google for: وزارة المواصلات قطر site:facebook.com
🔍 Searching Google for: وزارة المواصلات قطر site:instagram.com
🔍 Searching Google for: وزارة المواصلات قطر site:twitter.com
🔍 Searching Google for: وزارة البلدية قطر site:facebook.com
❌ Error for query وزارة البلدية قطر site:facebook.com: The read operation timed out
🔍 Searching Google for: وزارة البلدية قطر site:instagram.com
❌ Error for query وزارة البلدية قطر site:inst

In [7]:

df1 = pd.read_excel("reddit_qatar_feedback.xlsx")
df2 = pd.read_excel("fb_ig_from_google.xlsx")
merged_df = pd.concat([df1, df2], ignore_index=True)
merged_df.to_excel("qatar_social_feedback_combined.xlsx", index=False)
print("✅ All data saved to final Excel file.")


✅ All data saved to final Excel file.
